In [ ]:
import random
import json
import torch
from ipynb.fs.full.nltk_utils import tokenize, stem, bag_of_words
from ipynb.fs.full.model import NeuralNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # check gpu support 
with open('intents.json','r') as f:
    intents = json.load(f)
FILE = "data.pth"
data = torch.load(FILE)



input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]  
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]
model = NeuralNet(input_size, hidden_size, output_size).to(device) 
model.load_state_dict(model_state)
model.eval()


#bot

bot_name = "Sam"
print("Let's chat! type 'quit' to exit")
while True:
    sentence = input('You: ')
    if sentence == 'quit':
        break
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    output = model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]
    
    
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    
    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand..." ) 
                      

Let's chat! type 'quit' to exit
You: hi
Sam: Hi there, what can I do for you?
You: 
Sam: I do not understand...
You: hello 
Sam: Hello, thanks for visiting
You: what do you sell
Sam: We sell coffee and tea
You: can i have tea
Sam: We accept most major credit cards, and Paypal
You: bye
Sam: Have a nice day
You: good day
Sam: Hi there, how can I help?
You: hello
Sam: Hi there, what can I do for you?
You: hi
Sam: Hi there, how can I help?
You: hello
Sam: Hi there, how can I help?
You: bye'
Sam: Bye! Come back again soon.
You: goodbye
Sam: See you later, thanks for visiting
You: can i have
Sam: We accept VISA, Mastercard and Paypal
You: tell me a joke
Sam: What did the buffalo say when his son left for college? Bison.
You: funny
Sam: I do not understand...
